## 背景

FancyDsp中的大部分的品牌会利用秒针做三方监控，一来可以用来与客户数据进行三方对比，另一方面也可以监控我们自己的系统数据是否出现大的异常，在此情景下，AMO中也有一份对应的报表，抓取秒针的数据，并与FancyDSP的系统数据合并成一份报表。于是就会偶尔出现两边数据GAP过大的情况，需要排查清楚具体原因，减少执行风险
曝光gap通常有如下情况：

1. fancydsp与fancy秒针gap
2. fancydsp与客户秒针gap

## 报表的业务技术层面的知识

### 以下为基本相关业务知识

1. adid 会对应一个曝光监控链接与点击监控链接
2. adid 会有地域定向
3. 一般都是分订单监控，一个订单下有多个adid
4. 秒针的监控连接中有两个重要参数 （cusid, spid)
   + cusid在秒针中叫活动，一般是一个订单会对应一个cusid
   + spid是一个子类别，一个订单一个只有一个cusid, 也可以有多个spid，它可以用来区分不同地域定向，媒体定向等，根据实际情况约定使用
   + 一个adid一般只会有一个对应的spid，但反过来一个spid可以被多个adid使用

### 以下为AMO报表关联的基本逻辑

1. 系统会从秒针那边跑数据，可以分 cusid, spid, 城市，省份，媒体，platform等信息
2. 通过地域，订单，spid等信息与系统的数据关联起来，这里实际逻辑会比较复杂，暂时就按以下的核心逻辑来理解
   + city + spid + cusid 做关连，并且city包含在platform中

## 目前发现的原因有

配置原因：

1. 广告配置是忘开启地域定向，造成数据GAP
2. 监控连接挂错或忘挂，造成数据GAP
3. 素材的监控连接发生过修改，但是投放信息中依旧使用的是老的版本
4. （猜想原因）广告配置时媒体信息出错（这个原因只有在基于媒体分别监测时生效）。FTX三方DSP也会有挂错监测连接的情况

地域漂移原因：

1. IP竞价与曝光时不一致（直接地域漂移），造成数据GAP
2. IP库更新不及时，造成在数据上看起来FancyDSP与三方的地域不匹配，误认为IP飘移

媒体上报原因：

1. 曝光上报超时导致没有上报全部的监测，曝光检测上报通常是有顺序的，一般为FTX监测-DSP监测-客户秒针监测-fancy秒针监测-impsdk监测(仅qtt、韩剧有)
2. 参数替换失败导致，被客户或秒针系统过滤

## 排查步骤

### 1. 获取背景信息

1. 广告信息
   + 广告主
   + 订单号
   + 异常GAP的adgroup/adid信息
   + 对应的监测链接，秒针的话核心为cusid跟spid
   + 是否地域定向
   + 是否有媒体定向
2. 背景信息
   + 日期
   + 哪些点的GAP较大
     - 比如对应的媒体信息
     - 对应的地域信息
     - 对应的cusid, spid信息

### 2. 确认adid的监测链接挂载

1. 用得到的cusid, spid从数据库中得到adid列表
2. 与AE提供的adid列表对比，看是否有差异
3. 观察下广告名字中定义的地域，媒体等信息是否一致
   - 这个是19年年初的时候就出现了个小问题，实际生效的监测连接与运营设想的不一致（核心为批量创建出现的BUG）

> 这一步基本可以发现配置原因2，3两点，另外步骤4可以补充配置原因2

### 3. 通过对应adid曝光数据对应的竞价时的地域确认地域定向

如果有明确的地域定向要求的话，那可以从竞价日志中的地域信息判定定向问题

### 4. 对比对应adid的竞价地域与曝光地域，确认漂移问题的影响度

这里的影响度指的是竞价与曝光地域匹配的曝光数与总曝光数的比值

### 5. 对比对应adid在Fancy中的总曝光与秒针对应(cusid+spid)的总曝光

结合第4步与第5步可以确认配置原因2与地域漂移原因1

- 假如两边总曝光的GAP较小，那么就排查了原因2
- 假如同时漂移的影响度较高，则基本可确认IP的地域漂移带来数据GAP

### 6. 分媒体看曝光数据，是否出现其它媒体

这个可以确认猜想原因5。这个只有在spid有媒体定向的时候才有效，不然就没必要看了

### 7. 可以再次确认下地域GAP的问题，当然这一步放一开始就做，用于确认问题是真的存在的

In [1]:
# 准备工作
import numpy  as np
import pandas as pd
#连接presto库
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *

# 本地则使用代理，默认socks5端口号为18888
import os
if os.environ.get('JUPYTERENV') != 'release':
    import socks
    import socket
    socks.set_default_proxy(socks.SOCKS5, "localhost", 18888)
    socket.socket = socks.socksocket

presto_engine = create_engine('presto://192.168.28.111:10080/hive/dwd')

运营提供数据信息
![客户数据对比](../imgs/impgap/impgap.png)
根据表中数据，我方与秒针总数数无gap，三方认为我方数据有部分地域漂移。

地域漂移的原因可能有三个：
1. 因策略路由导致的（我们常称之为IP漂移）。
2. 各方IP库不同，或者IP库同步不一致。
3. 运营执行配置失误，造成地域定向出错。

In [4]:
#查询我方实际地域分布情况
#父订单id：44733，44732 投放周期 2021-03-11至2021-03-21 地域定向：版纳，德宏
impgap_df = pd.read_sql('''
select ext_parent_order_id,ext_geo_id,geo_code,b.area_name,count(1)
from dwd.d_ad_ftx_impression_data a
join dim.dim_area b on b.geo_b_id=cast(a.geo_code as varchar)
where thisdate between '2021-03-11' and '2021-03-21'
and ext_parent_order_id in (44733,44732)
group by 1,2,3,4
''', presto_engine)
impgap_df

,ext_parent_order_id,ext_geo_id,geo_code,area_name,_col4
0,44732,156024000011,156024000000,云南省,1
1,44733,156024000013,156024000013,德宏傣族景颇族自治州,19508
2,44732,156024000011,156001000000,北京,1
3,44733,156024000013,156024000003,大理白族自治州,2
4,44733,156024000013,156024000001,昆明,7
5,44732,156024000011,156012000004,廊坊,9
6,44732,156024000011,156024000011,西双版纳傣族自治州,20047


In [4]:
#根据子订单id：44767，任意投放日期：2021-03-16，从抽样表中查看fancy返回给媒体的数据
imp_df = pd.read_sql('''
select response_data from ods.o_ad_ftx_raw_log
where thisdate = '2021-03-16'
--  and dsp_response_data like '%%248237%%'
--   and dsp_sample_id = '1'
--  and vendor_id=120
and order_id in (44767)
limit 10
''', presto_engine)
imp_df.head()

,response_data
0,"{""ua"":""Mozilla%2f5.0%20%28Linux%3b%20Android%2..."
1,"{""ua"":""Mozilla%2f5.0%20%28Linux%3b%20Android%2..."
2,"{""ua"":""Mozilla%2f5.0%20%28iPhone%3b%20CPU%20iP..."
3,"{""ua"":""Mozilla%2f5.0%20%28Linux%3b%20Android%2..."
4,"{""ua"":""Mozilla%2f5.0%20%28Linux%3b%20Android%2..."


![抽样监测连接](../imgs/impgap/impgap2.png)
解析上述json，可以看到返回给媒体的曝光监测共三条，第一条为fancy的，其余为客户的。

根据查询数据，
1. 我方数据在请求地域(ext_geo_id)与曝光地域(geo_code)是一致的，无地域漂移现象，其次所投地域满足客户要求，所以运营配置是没有问题的。
2. 在与引擎沟通后，了解到月初已与秒针同步更新过ip库。在抽样数据中可以看到监测里替换的ip与我方ip一致(秒针可能使用自己获取的ip进行地域判断)
3. 客户使用的秒针监测域名为 https://tytx.m.cn.miaozhen.com 和 https://pd.ads.cn.miaozhen.com 其服务器所在的地域与我方服务器所在地域不一致，可能导致获取到的ip不同，从而出现ip漂移。

结论：因策略路由导致的（我们常称之为IP漂移）地域漂移现象是无法避免的，按照当前的gap情况看，gap大概为5%左右，若要完成客户排期，需要运营超额完成排期指标。